In [29]:
# data from: https://www.kaggle.com/datasets/bahadoreizadkhah/face-mask-types-dataset/code
# reference: https://towardsdatascience.com/from-alexnet-to-nasnet-a-brief-history-and-introduction-of-convolutional-neural-networks-cf63bf3320e1 
# reference: https://github.com/Alexiush/weak-causality-and-causal-disposition-in-images/blob/main/ca_cnn_sample_resnet.ipynb

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras import models
from sklearn.metrics import confusion_matrix
import pathlib

In [30]:
data_train = pathlib.Path('./Dataset/train')
data_test = pathlib.Path('./Dataset/test')

INPUT_SIZE = [300, 300]
BATCH = 6

In [31]:
training_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2, 
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.0,
    # rescale=1./255,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input
    )

training_data = training_datagen.flow_from_directory('./Dataset/train', 
                                                     subset="training", 
                                                     shuffle=True, 
                                                     target_size=INPUT_SIZE, 
                                                     batch_size=BATCH, 
                                                     color_mode="rgb",
                                                     class_mode='categorical')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    # rescale=1./255,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input
)

test_data = test_datagen.flow_from_directory('./Dataset/test', 
                                             target_size=INPUT_SIZE, 
                                             color_mode="rgb", 
                                             batch_size=BATCH,
                                             class_mode='categorical',
                                             shuffle=False)

# display dimensions
x_test, y_test = next(test_data)
x_train, y_train = next(training_data)

Found 1956 images belonging to 5 classes.
Found 330 images belonging to 5 classes.


In [32]:
# pooling='avg' => same as adding GlobalAveragePooling2D 
resn_model = tf.keras.applications.ResNet101V2(input_shape=(300, 300,3), include_top=False, weights='imagenet')
resn_model.trainable = True

fine_tune_at = 100
for layer in resn_model.layers[:fine_tune_at]:
    layer.trainable = False

resn_model.summary()

In [33]:
import tensorflow as tf

tf.keras.utils.plot_model(resn_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [34]:
import visualkeras

visualkeras.layered_view(resn_model)

In [35]:
model = models.Sequential([# Lambda(lambda x: x, input_shape=(300, 300, 3)), 
    tf.keras.layers.InputLayer((300, 300, 3)),
    resn_model,
    GlobalAveragePooling2D(),
    Flatten(),
    tf.keras.layers.Dropout(0.5),
    Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    Dense(5, activation='softmax')])

# model = models.Sequential([# Lambda(lambda x: x, input_shape=(300, 300, 3)), 
#     tf.keras.layers.InputLayer((300, 300, 3)),
#     resn_model,
#     tf.keras.layers.AveragePooling2D(pool_size = (4,4), padding='same'),
#     Flatten(),
#     Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     Dense(5, activation='softmax')])
# 
# model.summary()

In [36]:
tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [37]:
visualkeras.layered_view(model)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # 'adam'
    metrics=['accuracy']
)
#  optimizer=tf.keras.optimizers.SGD(learning_rate=0.01)

history = model.fit(
    training_data,
    validation_data=test_data,
    # x=x_train,
    # y=y_train,
    # validation_split=0.2,
    epochs=10,
    batch_size=BATCH,
    # steps_per_epoch=len(training_data),
    # validation_steps=len(test_data)
)

Epoch 1/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 477s 1s/step - accuracy: 0.2343 - loss: 1.7543 - val_accuracy: 0.4485 - val_loss: 1.1906
Epoch 2/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 462s 1s/step - accuracy: 0.3942 - loss: 1.3854 - val_accuracy: 0.6818 - val_loss: 0.9888
Epoch 3/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 472s 1s/step - accuracy: 0.5334 - loss: 1.1273 - val_accuracy: 0.7758 - val_loss: 0.7555
Epoch 4/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 496s 2s/step - accuracy: 0.6192 - loss: 0.9615 - val_accuracy: 0.7303 - val_loss: 0.8829
Epoch 5/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 482s 1s/step - accuracy: 0.6770 - loss: 0.8907 - val_accuracy: 0.9000 - val_loss: 0.4325
Epoch 6/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 477s 1s/step - accuracy: 0.7369 - loss: 0.7786 - val_accuracy: 0.9152 - val_loss: 0.4751
Epoch 7/10
274/326 ━━━━━━━━━━━━━━━━━━━━ 1:12 1s/step - accuracy: 0.7614 - loss: 0.7154

In [ ]:
# create confusion matrix
y_pred=model.predict(test_data, batch_size=BATCH)
print(y_pred.shape)

y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

cm=confusion_matrix(test_data.classes, y_pred)
print(cm)

#Impresion de la exactitud
accu = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
accu =accu.diagonal()
print(accu)

loss, accuracy = model.evaluate(test_data, verbose=0)
print(loss, accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(test_data.classes, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=['cloth', 'n95', 'n95v', 'nfm', 'srg'], yticklabels=['cloth', 'n95', 'n95v', 'nfm', 'srg'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix Data Train')
plt.show()

In [ ]:
# plot the loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
# plt.savefig('AccVal_acc')

In [ ]:
import numpy as np

test_image = tf.keras.preprocessing.image.load_img(
    './Dataset/test/cloth/1.jpg',
    target_size=INPUT_SIZE)
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = test_image / 255
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)
i = np.argmax(result[0])
classes = training_data.class_indices.keys()
print(classes)
list(classes)[i]

# if result[0][0] < 0.5:
#     print('Image classified as WithMAsk')
# else:
#     print('Image classified as Without Mask')

In [ ]:
def plot_image(j, predictions_array, true_label, img):
    true_label, img = true_label[j], img[j]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    plt.imshow(img, cmap=plt.cm.binary)
    
    predicted_label = np.argmax(predictions_array)
    if predicted_label == np.argmax(true_label):
        color = 'blue'
    else:
        color = 'red'
    
    plt.xlabel("{} {:2.0f}% ({})".format(classes[predicted_label],
                                         100*np.max(predictions_array),
                                         classes[np.argmax(true_label)]),
               color=color)

def plot_value_array(k, predictions_array, true_label):
    true_label = true_label[k]
    plt.grid(False)
    plt.xticks(range(5))
    plt.yticks([])
    thisplot = plt.bar(range(5), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[np.argmax(true_label)].set_color('blue')

i = 1
print(classes)
print(classes[i])

plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, result, y_test, x_test)
plt.subplot(1,2,2)
plot_value_array(i, result[0], y_test)
plt.show()